# Import statements + data import
- Numpy for linear algebra
- Pandas for dataset manips
- 

In [1]:
import numpy as np
import pandas as pd
from time import perf_counter
import scipy.linalg as la

In [2]:
filename = "Matrix_Data_Export.csv"
data_set = pd.read_csv(filename)

## Matrix + Process Setup:

- `interaction_constant(A,i,j)`: calculate our interaction constant and update matrix `M`

    - Inputs:
        - `M`: Matrix for which to update interaction constant at location `(i,j)`
        - `i,j`: Integers that denote our parameters to calculate the interaction for; `i` is independent, `j` is dependent
    - Outputs:
        - Returns our LSRL vector $[\beta_0, \beta_1]^T$ where $y = \beta_0 + \beta_1 x$
        - non-pure function (side effects on `M`)
    - Notes:

#### Parameters of interest -> Pandas Labels:

1. Relative Humidity ->  RH(%)
2. Temperature -> T(degC)
3. Light VOCs -> LightVOC(ADU)
4. Heavy VOCs -> HeavyVOC(ADU) 
5. OZONE -> Ozone(ADU)
6. CO -> CO(ADU)
7. CO_2 -> CO2(ADU)
8. Elevation -> Elevation(ft)
9. Distance from nearest highway -> Distance From Road (miles)

In [3]:
parameters = ['RH(%)', 'T(degC)', 'LightVOC(ADU)', 'HeavyVOC(ADC)', 'Ozone(ADU)', 'CO(ADU)', 'CO2(ADU)', 'Elevation(ft)', 'Distance From Road (miles)']

def interaction_constant(M, i,j):
    x = np.array(data_set[parameters[i]])
    y = np.array(data_set[parameters[j]])
    b = y
    A = [[np.float64(1.0), x[k]] for k in range(len(x))]
    Q, R = np.linalg.qr(A)
    Q_t = np.linalg.matrix_transpose(Q)
    R_inv = np.linalg.inv(R)
    x = np.matmul(np.matmul(R_inv, Q_t), b)
    M[i][j] = x[1]
    # M[j][i] = x[1]
    return x

## Interaction Matrix

In [4]:
interaction_matrix = np.identity(10) 
row_labels = parameters + ['Row sum']
col_labels = parameters + ['Col sum']

for i in range(9):
    for j in range(9):
        interaction_constant(interaction_matrix, i,j)

for i in range(10):
    interaction_matrix[9][i] = sum(interaction_matrix[:,i])
    interaction_matrix[i][9] = sum(interaction_matrix[i])
            
df = pd.DataFrame(interaction_matrix, index=col_labels, columns=row_labels)
df.to_csv('interactions.csv')


## Ill Condition Showcase

Below we print the condition number for each independent RV

In [5]:
for i in range(9):
    temp_data = np.array(data_set[parameters[i]])
    matrix_A = [[np.float64(1.0), temp_data[k]] for k in range(len(temp_data))]
    cond = np.linalg.cond(matrix_A)
    print(f"Parameter {parameters[i]} condition number: {cond}")

Parameter RH(%) condition number: 59.184956958437525
Parameter T(degC) condition number: 28.142512126691678
Parameter LightVOC(ADU) condition number: 11000.490705931266
Parameter HeavyVOC(ADC) condition number: 2433.147821091631
Parameter Ozone(ADU) condition number: 13016.21585877661
Parameter CO(ADU) condition number: 19833.588411993394
Parameter CO2(ADU) condition number: 880.8290582906966
Parameter Elevation(ft) condition number: 209797.42882930793
Parameter Distance From Road (miles) condition number: 5.150658368712972


## Speed Difference in LU vs QR

- `interaction_constant_LU(A,i,j)`: calculate our interaction constant and update matrix `M` with LU decomposition instead of QR
    - Specifically, we solve system $\mathbf{A}^T\overrightarrow{\mathbf{b}} = \mathbf{A}^T\mathbf{A}\overrightarrow{\mathbf{x}}$
    
- This is not used in the report since numpy solve is less optimal than our QR code

In [6]:
def interaction_constant_LU(i,j):
    x = np.array(data_set[parameters[i]])
    y = np.array(data_set[parameters[j]])
    b = y
    A = [[np.float64(1.0), x[k]] for k in range(len(x))]
    A_t = np.linalg.matrix_transpose(A)
    r_vector = np.matmul(A_t, b)
    coeff = np.matmul(A_t, A)
    return np.linalg.solve(coeff,r_vector)

In [7]:
qr_times = []
lu_times = []

for i in range(1000):
    random = np.identity(7)
    t1qr = perf_counter()
    sol1 = interaction_constant(random, 5, 1)
    t2qr = perf_counter()

    t1lu = perf_counter()
    sol2 = interaction_constant_LU(5, 1)
    t2lu = perf_counter()
    
    qr_times.append(t2qr - t1qr)
    lu_times.append(t2lu - t1lu)

qr_times = np.array(qr_times)
lu_times = np.array(lu_times)

time_diff = qr_times - lu_times
avg_cost = sum(time_diff)/len(time_diff)
stdev1 = np.std(qr_times)
stdev2 = np.std(lu_times)
stdev = np.sqrt((stdev1**2 + stdev2**2)/len(time_diff))
print(f"Average Time Increase for QR: {avg_cost}")
print(f"Standard Deviation: {stdev}")

Average Time Increase for QR: -0.0002757604050248119
Standard Deviation: 3.503966704854704e-06
